In [3]:
import numpy.random as npr
import numpy as np
import math
def gen_sn(M, I, anti_paths=True, mo_match=True):
    ''' Function to generate random numbers for simulation.
    Parameters
    ==========
    M: int
    number of time intervals for discretization
    I: int
    number of paths to be simulated
    anti_paths: boolean
    use of antithetic variates
    mo_math: boolean
    use of moment matching
    '''
    if anti_paths is True:
        sn = npr.standard_normal((M + 1, int(I / 2)))
        sn = np.concatenate((sn, -sn), axis=1)
    else:
        sn = npr.standard_normal((M + 1, I))
    if mo_match is True:
        sn = (sn - sn.mean()) / sn.std()
    return sn
# ----------Tđịnh giá quyền chọn mua châu Âu (European call option) theo mô hình Black-Scholes-Merton (BSM) bằng cách sử dụng mô phỏng Monte Carlo.---
S0 = 100.
r = 0.05
sigma = 0.25
T = 1.0
I = 50000

def gbm_mcs_stat(K):
    ''' Valuation of European call option in Black-Scholes-Merton
    by Monte Carlo simulation (of index level at maturity)
    Parameters
    ==========
    K: float
    (positive) strike price of the option
    Return=======
    C0: float
    estimated present value of European call option
    '''
    sn = gen_sn(1, I)
    # simulate index level at maturity
    ST = S0 * np.exp((r - 0.5 * sigma ** 2) * T
    + sigma * math.sqrt(T) * sn[1])
    # calculate payoff at maturity
    hT = np.maximum(ST - K, 0)
    # calculate MCS estimator
    C0 = math.exp(-r * T) * np.mean(hT)
    return C0
ans = gbm_mcs_stat(105)
# Giá trị hiện tại ước tính của quyền chọn mua châu Âu: Kết quả của hàm gbm_mcs_stat là khoảng 10.04, 
# có nghĩa là giá trị hiện tại của quyền chọn mua châu Âu với giá thực hiện là 105 (K = 105) được ước tính là khoảng 10.04 đơn vị tiền tệ.


np.float64(9.985564665036)

In [4]:
# Mở rộng phương pháp định giá quyền chọn bằng Monte Carlo mô phỏng cho cả quyền chọn mua (call option) và quyền chọn bán (put option) của châu Âu. 
# Hàm gbm_mcs_dyna được sử dụng để mô phỏng các đường đi của giá 
# cổ phiếu theo thời gian và tính toán giá trị hiện tại của quyền chọn mua hoặc bán.
M = 50 
def gbm_mcs_dyna(K, option='call'):
    ''' Valuation of European options in Black-Scholes-Merton
    by Monte Carlo simulation (of index level paths)
    Parameters
    ==========
    K: float
    (positive) strike price of the option
    option : string
    type of the option to be valued ('call', 'put')
    Returns
    =======
    C0: float
    estimated present value of European call option
    '''
    dt = T / M
    # simulation of index level paths
    S = np.zeros((M + 1, I))
    S[0] = S0
    sn = gen_sn(M, I)
    for t in range(1, M + 1):
        S[t] = S[t - 1] * np.exp((r - 0.5 * sigma ** 2) * dt
        + sigma * math.sqrt(dt) * sn[t])
        # case-based calculation of payoff
    if option == 'call':
        hT = np.maximum(S[-1] - K, 0)
    else:
        hT = np.maximum(K - S[-1], 0)
    # calculation of MCS estimator
    C0 = math.exp(-r * T) * np.mean(hT)
    return C0
a = gbm_mcs_dyna(K=110., option='call') 
b = gbm_mcs_dyna(K=110., option='put') 
print(a)
print(b)
# Quyền chọn mua (Call Option): Quyền mua cổ phiếu với giá thực hiện trong tương lai. Bạn mua quyền chọn này nếu dự đoán giá cổ phiếu sẽ tăng.
# Quyền chọn bán (Put Option): Quyền bán cổ phiếu với giá thực hiện trong tương lai. Bạn mua quyền chọn này nếu dự đoán giá cổ phiếu sẽ giảm.

8.043782605986976
12.660707834952833


Chú ý :
Công Thức Black-Scholes-Merton (BSM):
    Công thức Black-Scholes-Merton (BSM) cung cấp một giải pháp chính xác và phân tích cho việc định giá quyền chọn châu Âu. Nó dựa trên một số giả định:

        Chuyển động Brown Hình học: Giá tài sản cơ sở theo một quá trình ngẫu nhiên với biến động đồng đều.
        Thị trường hoàn hảo: Không có chi phí giao dịch, và các tài sản có thể được mua hoặc bán mà không có ảnh hưởng đến giá.
        Lãi suất không rủi ro: Được giả định là không thay đổi theo thời gian.
        Độ biến động ổn định: Được giả định là không thay đổi theo thời gian.
Phương Pháp Mô Phỏng Monte Carlo
    Phương pháp mô phỏng Monte Carlo là một kỹ thuật dựa trên việc mô phỏng nhiều kịch bản ngẫu nhiên để ước lượng giá trị của quyền chọn. Nó không yêu cầu các giả định như công thức BSM, và do đó có thể được áp dụng cho các quyền chọn phức tạp hơn, chẳng hạn như quyền chọn kiểu Mỹ hoặc các quyền chọn có đặc điểm đặc biệt.

    Quy Trình Mô Phỏng Monte Carlo:
    Tạo các số ngẫu nhiên: Sinh ra các chuỗi số ngẫu nhiên để mô phỏng các kịch bản của giá tài sản cơ sở trong tương lai.
    Mô phỏng các đường đi của giá: Sử dụng các số ngẫu nhiên để tạo ra nhiều đường đi khác nhau của giá cổ phiếu.
    Tính toán giá trị quyền chọn: Đối với mỗi kịch bản, tính giá trị quyền chọn tại ngày đáo hạn.
    Tính giá trị kỳ vọng: Tính giá trị trung bình của các giá trị quyền chọn từ tất cả các kịch bản và chiết khấu về hiện tại để có được giá trị quyền chọn ước lượng.

So Sánh
-Công Thức BSM:

    Ưu điểm: Chính xác và nhanh chóng nếu các giả định của mô hình được thoả mãn.
    Nhược điểm: Chỉ áp dụng cho quyền chọn châu Âu và không thể xử lý các tình huống phức tạp như quyền chọn kiểu Mỹ hay các quyền chọn có đặc điểm đặc biệt.
-Phương Pháp Mô Phỏng Monte Carlo:

    Ưu điểm: Linh hoạt và có thể xử lý các quyền chọn phức tạp và điều kiện không tiêu chuẩn.
    Nhược điểm: Có thể yêu cầu tính toán lớn và thời gian chạy lâu hơn, đặc biệt là khi số lượng mô phỏng cần thiết cao.
Kết Luận
Công thức BSM cung cấp một cách nhanh chóng và chính xác để định giá quyền chọn châu Âu khi các giả định của mô hình được thoả mãn.
Phương pháp mô phỏng Monte Carlo cung cấp sự linh hoạt trong việc định giá các quyền chọn phức tạp và không yêu cầu các giả định mạnh mẽ như công thức BSM.